In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import math
import json
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from IPython.display import Image
import seaborn as sns
import missingno as msnco

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon/test.csv')

In [ ]:
sample_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon/sample_submission.csv')

In [ ]:
train_df.drop(['Cancelled','Diverted'], axis=1, inplace= True) #Cancelled와 Diverted 데이터는 모든 데이터가 0으로 존재 --> drop 

In [ ]:
#delay 1, 0 으로 변환
dic = {'Not_Delayed' : 0, 'Delayed' : 1}
train_df['Delay'] = train_df['Delay'].map(dic) # delay float타입으로 변경

In [ ]:
df = train_df[['ID','Estimated_Departure_Time','Estimated_Arrival_Time','Origin_State','Destination_State','Distance','Delay']]

In [ ]:
df = df.dropna(subset=['Origin_State','Destination_State']) #결측치 제거 

In [ ]:
def time_diff(df,state):
    destination_list =  list(set(df[df['Origin_State']==state]['Destination_State'].values.tolist()))
    time_diff_dict = {}
    for d in destination_list:
        a = df[(df['Origin_State']==state) & (df['Destination_State']==d)] #ali 출발 , texas 도착
        b = df[(df['Origin_State']==d) & (df['Destination_State']==state)] #texas 출. ali 도착
        a_time = (a['Estimated_Arrival_Time']-a['Estimated_Departure_Time']).mean()     
        b_time = (b['Estimated_Arrival_Time']-b['Estimated_Departure_Time']).mean()
        time_di = (a_time - b_time)/2 #a_state와 b_state 의 시차  --> 출발a 도착b or 출발b 도착a인 데이터에 시차 컬럼 넣어주기 
        time_di = round(time_di)
        time_diff_dict[state, d] = time_di
    
    return time_diff_dict

In [ ]:
time_delay = dict() #시간 결측치가 존재하는 데이터의 경우 오류발생
for i in state_list:
    time_delay.update(time_diff(df, i))
print(time_delay)

In [ ]:
def find_time_lag(df, state):
    OA = df[df['Origin_State'] == state]
    predict_st = OA.Destination_State.unique().tolist()
    DA = df[df['Destination_State'] == state]
    time_delay_dict = {}
    for st in predict_st:
        # if train_df = train_df[train_df['Estimated_Arrival_Time'] > train_df['Estimated_Departure_Time']]: #도착시간 > 출발시간 
        st_oa = OA[OA['Destination_State'] == st]
        st_oa = st_oa[st_oa['Estimated_Arrival_Time'] > st_oa['Estimated_Departure_Time']]
        st_da = DA[DA['Origin_State'] == st]
        st_da = st_da[st_da['Estimated_Arrival_Time'] > st_da['Estimated_Departure_Time']]
        st_oa_diff = (st_oa.Estimated_Arrival_Time - st_oa.Estimated_Departure_Time).mean()
        st_da_diff = (st_da.Estimated_Arrival_Time - st_da.Estimated_Departure_Time).mean()
        time_delay = st_da_diff - st_oa_diff
        if not np.isnan(time_delay):
            time_delay_dict[state, st] = -round(time_delay/2/60)
    
    return time_delay_dict

In [ ]:
time_delay = dict()
for i in train_df['Origin_State'].unique().tolist():
    time_delay.update(find_time_lag(train_df, i))
print(time_delay)

{('Oklahoma', 'Texas'): 0, ('Oklahoma', 'Colorado'): -2, ('Oklahoma', 'Georgia'): 1, ('Oklahoma', 'Illinois'): 0, ('Oklahoma', 'New Jersey'): 1, ('Oklahoma', 'California'): -3, ('Oklahoma', 'Virginia'): 1, ('Oklahoma', 'Florida'): 1, ('Oklahoma', 'Missouri'): 0, ('Oklahoma', 'Nevada'): -3, ('Oklahoma', 'North Carolina'): 2, ('Oklahoma', 'Minnesota'): 0, ('Oklahoma', 'Arizona'): -2, ('Oklahoma', 'Utah'): -1, ('Oklahoma', 'Maryland'): 1, ('Oklahoma', 'Michigan'): 1, ('Oklahoma', 'Pennsylvania'): 1, ('Oklahoma', 'Washington'): -3, ('Illinois', 'Utah'): -1, ('Illinois', 'Missouri'): 0, ('Illinois', 'Kentucky'): 1, ('Illinois', 'Tennessee'): 0, ('Illinois', 'Ohio'): 2, ('Illinois', 'Minnesota'): 0, ('Illinois', 'West Virginia'): 2, ('Illinois', 'Michigan'): 2, ('Illinois', 'Oklahoma'): 0, ('Illinois', 'Washington'): -3, ('Illinois', 'New York'): 1, ('Illinois', 'Texas'): 0, ('Illinois', 'California'): -3, ('Illinois', 'Indiana'): 1, ('Illinois', 'Illinois'): 0, ('Illinois', 'Pennsylvania'):

In [ ]:
len(time_delay.values())

In [ ]:
# OA = df[df['Origin_State'] == 'Oklahoma'] #출발지가 오클라호마
# predict_st = OA.Destination_State.unique().tolist() #츌벌자거 오클라호마인 비행편의 도착지 리스트
# DA = df[df['Destination_State'] == 'Oklahoma'] #도착지가 오클라호마
# time_delay_dict = {}
# st_oa = OA[OA['Destination_State'] == 'Texas'] #출발지가 오클라호마이고 도착지가 텍사스
# st_da = DA[DA['Origin_State'] == 'Texas']
# st_oa = st_oa[st_oa['Estimated_Arrival_Time'] > st_oa['Estimated_Departure_Time']]
# st_da = st_da[st_da['Estimated_Arrival_Time'] > st_da['Estimated_Departure_Time']]
# st_oa_diff = (st_oa.Estimated_Arrival_Time - st_oa.Estimated_Departure_Time).mean() #출발지가 오클라호마이고 도착지가 텍사스인 비행편의 도착시간- 출발시간 의 평균 --> 도착시간이 출발시간보다 큰 것만 계산하기
# st_da_diff = (st_da.Estimated_Arrival_Time - st_da.Estimated_Departure_Time).mean()
# time_delay = st_da_diff - st_oa_diff
# time_delay

In [ ]:
#if 출발==a , 도착==b :
#   df['time_diff'] = time_delay[a,b]

In [ ]:
time_delay['Oklahoma', 'Texas']

0

In [ ]:
state_list = df['Origin_State'].unique().tolist()
state_list #52 개주 

In [ ]:
df[df['Origin_State']=='Texas'].Destination_State.unique().tolist()

In [ ]:
for i in state_list:
    destination_state = df[df['Origin_State']==i].Destination_State.unique().tolist()
    for j in destination_state:
        if df['Origin_State']==i and df['Destination_State']==j :
            df['time_diff']=time_delay[i,j]

In [ ]:
df.columns

Index(['ID', 'Estimated_Departure_Time', 'Estimated_Arrival_Time',
       'Origin_State', 'Destination_State', 'Distance', 'Delay'],
      dtype='object')

In [ ]:
df

,ID,Estimated_Departure_Time,Estimated_Arrival_Time,Origin_State,Destination_State,Distance,Delay
0,TRAIN_000000,NaN,NaN,Oklahoma,Texas,419.0,NaN
1,TRAIN_000001,740.0,1024.0,Illinois,Utah,1250.0,NaN
2,TRAIN_000002,1610.0,1805.0,North Carolina,New York,544.0,NaN
3,TRAIN_000003,905.0,1735.0,California,New Jersey,2454.0,NaN
4,TRAIN_000004,900.0,1019.0,California,California,250.0,NaN
...,...,...,...,...,...,...,...
999990,TRAIN_999990,NaN,751.0,Illinois,Texas,802.0,NaN
999991,TRAIN_999991,1720.0,1838.0,Virginia,New York,250.0,NaN
999992,TRAIN_999992,1307.0,1707.0,Wyoming,Illinois,1162.0,0.0
999993,TRAIN_999993,603.0,755.0,Virginia,Colorado,1452.0,NaN


1. 시차 time_delay 
2. df에 시차 컬럼 추가 
3. 시차를 고려한 비행시간 컬럼 추가 
4. 도착시간-출발시간으로 구한 비행시간과 평균비행시간의 비교

0425

In [ ]:
df = df[df['Estimated_Arrival_Time']>df['Estimated_Departure_Time']]
df

,ID,Estimated_Departure_Time,Estimated_Arrival_Time,Origin_State,Destination_State,Distance,Delay
1,TRAIN_000001,740.0,1024.0,Illinois,Utah,1250.0,NaN
2,TRAIN_000002,1610.0,1805.0,North Carolina,New York,544.0,NaN
3,TRAIN_000003,905.0,1735.0,California,New Jersey,2454.0,NaN
4,TRAIN_000004,900.0,1019.0,California,California,250.0,NaN
6,TRAIN_000006,1742.0,1903.0,New Jersey,Massachusetts,200.0,0.0
...,...,...,...,...,...,...,...
999989,TRAIN_999989,1717.0,2019.0,New Jersey,Florida,1023.0,NaN
999991,TRAIN_999991,1720.0,1838.0,Virginia,New York,250.0,NaN
999992,TRAIN_999992,1307.0,1707.0,Wyoming,Illinois,1162.0,0.0
999993,TRAIN_999993,603.0,755.0,Virginia,Colorado,1452.0,NaN


In [ ]:
df.isna().sum()

ID                               0
Estimated_Departure_Time         0
Estimated_Arrival_Time           0
Origin_State                     0
Destination_State                0
Distance                         0
Delay                       453129
dtype: int64

In [ ]:
state_list = df['Origin_State'].unique().tolist()

In [ ]:
def time_diff(df,state):
    destination_list =  df[df['Origin_State']==state]['Destination_State'].unique().tolist() #출발주가 state인 데이터의 도착주 리스트 
    time_diff_dict = {} #딕셔너리 
    for d in destination_list: 
        a = df[(df['Origin_State']==state) & (df['Destination_State']==d)] #ali 출발 , texas 도착
        b = df[(df['Origin_State']==d) & (df['Destination_State']==state)] #texas 출. ali 도착
        a_time = (a['Estimated_Arrival_Time']-a['Estimated_Departure_Time']).mean()     
        b_time = (b['Estimated_Arrival_Time']-b['Estimated_Departure_Time']).mean()
        time_di = round((a_time - b_time)/2) #a_state와 b_state 의 시차  --> 출발a 도착b or 출발b 도착a인 데이터에 시차 컬럼 넣어주기 
        time_diff_dict[state, d] = time_di
    
    return time_diff_dict

In [ ]:
a = df[(df['Origin_State']=='Texas') & (df['Destination_State']=='California') ]
b = df[(df['Origin_State']=='California') & (df['Destination_State']=='Texas') ]
a_time = (a['Estimated_Arrival_Time']-a['Estimated_Departure_Time']).mean() 
b_time = (b['Estimated_Arrival_Time']-b['Estimated_Departure_Time']).mean()
time_di = round((a_time-b_time)/2)
time_di 
# time = round(time
# time

-180

In [ ]:
time_delay = dict() #시간 결측치가 존재하는 데이터의 경우 오류발생
for i in state_list:
    time_delay.update(time_diff(df, i))
print(time_delay)

In [ ]:
def find_time_lag(train_df, state):
    OA = train_df[train_df['Origin_State'] == state]  #출발 state a
    predict_st = OA.Destination_State.unique().tolist() #출발 state a의 도착 state 리스트 //  destination_list와 동일 
    DA = train_df[train_df['Destination_State'] == state] #도착 state a
    time_delay_dict = {}
    for st in predict_st:
        st_oa = OA[OA['Destination_State'] == st] #출발a , 도착[1,2,3,4]  for문 
        st_oa_diff = st_oa.Estimated_Arrival_Time.mean() - st_oa.Estimated_Departure_Time.mean()
        st_da = DA[DA['Origin_State'] == st]
        st_da_diff = st_da.Estimated_Arrival_Time.mean() - st_da.Estimated_Departure_Time.mean()
        time_delay = st_da_diff - st_oa_diff
        if not np.isnan(time_delay):
            time_delay_dict[state, st] = -round(time_delay/2/60)
    
    return time_delay_dict

In [ ]:
time_delay = dict()
for i in train_df['Origin_State'].unique().tolist():
    time_delay.update(find_time_lag(train_df, i))
print(time_delay)

{('Oklahoma', 'Texas'): 0, ('Oklahoma', 'Colorado'): -2, ('Oklahoma', 'Georgia'): 1, ('Oklahoma', 'Illinois'): 0, ('Oklahoma', 'New Jersey'): 1, ('Oklahoma', 'California'): -3, ('Oklahoma', 'Virginia'): 1, ('Oklahoma', 'Florida'): 1, ('Oklahoma', 'Missouri'): 0, ('Oklahoma', 'Nevada'): -1, ('Oklahoma', 'North Carolina'): 2, ('Oklahoma', 'Minnesota'): 0, ('Oklahoma', 'Arizona'): -2, ('Oklahoma', 'Utah'): -2, ('Oklahoma', 'Maryland'): 1, ('Oklahoma', 'Michigan'): 1, ('Oklahoma', 'Pennsylvania'): 1, ('Oklahoma', 'Washington'): -3, ('Illinois', 'Utah'): -1, ('Illinois', 'Missouri'): 0, ('Illinois', 'Kentucky'): 0, ('Illinois', 'Tennessee'): 0, ('Illinois', 'Ohio'): 0, ('Illinois', 'Minnesota'): 0, ('Illinois', 'West Virginia'): 2, ('Illinois', 'Michigan'): 1, ('Illinois', 'Oklahoma'): 0, ('Illinois', 'Washington'): 0, ('Illinois', 'New York'): 1, ('Illinois', 'Texas'): 0, ('Illinois', 'California'): 0, ('Illinois', 'Indiana'): 1, ('Illinois', 'Illinois'): 0, ('Illinois', 'Pennsylvania'): 0

In [ ]:
state_list = df['Origin_State'].unique().tolist()

In [ ]:
def time_diff(df,state):
    destination_list =  df[df['Origin_State']==state]['Destination_State'].unique().tolist() #출발주가 state인 데이터의 도착주 리스트 
    time_diff_dict = {} #딕셔너리 
    for d in destination_list: 
        a = df[(df['Origin_State']==state) & (df['Destination_State']==d)] #ali 출발 , texas 도착
        b = df[(df['Origin_State']==d) & (df['Destination_State']==state)] #texas 출. ali 도착
        a_time = (a['Estimated_Arrival_Time']-a['Estimated_Departure_Time']).mean()     
        b_time = (b['Estimated_Arrival_Time']-b['Estimated_Departure_Time']).mean()
        time_di = round((a_time - b_time)/2) #a_state와 b_state 의 시차  --> 출발a 도착b or 출발b 도착a인 데이터에 시차 컬럼 넣어주기 
        time_diff_dict[state, d] = time_di
    
    return time_diff_dict

In [ ]:
time_delay = dict() #시간 결측치가 존재하는 데이터의 경우 오류발생
for i in state_list:
    time_delay.update(time_diff(df, i))
print(time_delay)

ValueError: ignored

-

---



In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import math
import json
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from IPython.display import Image
import seaborn as sns
import missingno as msnco

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon/test.csv')

In [ ]:
sample_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon/sample_submission.csv')

In [ ]:
train_df.drop(['Cancelled','Diverted'], axis=1, inplace= True) #Cancelled와 Diverted 데이터는 모든 데이터가 0으로 존재 --> drop 

In [ ]:
#delay 1, 0 으로 변환
dic = {'Not_Delayed' : 0, 'Delayed' : 1}
train_df['Delay'] = train_df['Delay'].map(dic) # delay float타입으로 변경

In [ ]:
train_df

In [ ]:
df = train_df[['ID','Estimated_Departure_Time','Estimated_Arrival_Time','Origin_State','Destination_State','Distance','Delay']]

In [ ]:
df = df.dropna(subset=['Origin_State','Destination_State']) #결측치 제거 

In [ ]:
df = df[df['Estimated_Arrival_Time']>df['Estimated_Departure_Time']] #도착시간이 출발시간보다 큰 경우만 
df

,ID,Estimated_Departure_Time,Estimated_Arrival_Time,Origin_State,Destination_State,Distance,Delay
1,TRAIN_000001,740.0,1024.0,Illinois,Utah,1250.0,NaN
2,TRAIN_000002,1610.0,1805.0,North Carolina,New York,544.0,NaN
3,TRAIN_000003,905.0,1735.0,California,New Jersey,2454.0,NaN
4,TRAIN_000004,900.0,1019.0,California,California,250.0,NaN
6,TRAIN_000006,1742.0,1903.0,New Jersey,Massachusetts,200.0,0.0
...,...,...,...,...,...,...,...
999995,TRAIN_999995,936.0,1243.0,NaN,NaN,678.0,NaN
999996,TRAIN_999996,920.0,1028.0,NaN,Minnesota,223.0,NaN
999997,TRAIN_999997,800.0,1340.0,NaN,Texas,1642.0,NaN
999998,TRAIN_999998,1613.0,1824.0,Tennessee,NaN,214.0,NaN


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 766457 entries, 1 to 999999
Data columns (total 7 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        766457 non-null  object 
 1   Estimated_Departure_Time  766457 non-null  float64
 2   Estimated_Arrival_Time    766457 non-null  float64
 3   Origin_State              683174 non-null  object 
 4   Destination_State         682779 non-null  object 
 5   Distance                  766457 non-null  float64
 6   Delay                     195759 non-null  float64
dtypes: float64(4), object(3)
memory usage: 46.8+ MB


In [ ]:
def find_time_lag(train_df, state):
    OA = train_df[train_df['Origin_State'] == state]  #출발 state a
    predict_st = OA.Destination_State.unique().tolist() #출발 state a의 도착 state 리스트 //  destination_list와 동일 
    DA = train_df[train_df['Destination_State'] == state] #도착 state a
    time_delay_dict = {}
    for st in predict_st:
        st_oa = OA[OA['Destination_State'] == st] #출발a , 도착[1,2,3,4]  for문 
        st_oa_diff = st_oa.Estimated_Arrival_Time.mean() - st_oa.Estimated_Departure_Time.mean()
        st_da = DA[DA['Origin_State'] == st]
        st_da_diff = st_da.Estimated_Arrival_Time.mean() - st_da.Estimated_Departure_Time.mean()
        time_delay = st_da_diff - st_oa_diff
        if not np.isnan(time_delay):
            time_delay_dict[state, st] = -round(time_delay/2)
    
    return time_delay_dict

In [ ]:
time_delay = dict()
for i in train_df['Origin_State'].unique().tolist():
    time_delay.update(find_time_lag(train_df, i))
print(time_delay)

{('Oklahoma', 'Texas'): 10, ('Oklahoma', 'Colorado'): -99, ('Oklahoma', 'Georgia'): 87, ('Oklahoma', 'Illinois'): 14, ('Oklahoma', 'New Jersey'): 73, ('Oklahoma', 'California'): -170, ('Oklahoma', 'Virginia'): 66, ('Oklahoma', 'Florida'): 74, ('Oklahoma', 'Missouri'): -19, ('Oklahoma', 'Nevada'): -33, ('Oklahoma', 'North Carolina'): 103, ('Oklahoma', 'Minnesota'): 3, ('Oklahoma', 'Arizona'): -119, ('Oklahoma', 'Utah'): -103, ('Oklahoma', 'Maryland'): 69, ('Oklahoma', 'Michigan'): 49, ('Oklahoma', 'Pennsylvania'): 39, ('Oklahoma', 'Washington'): -176, ('Illinois', 'Utah'): -64, ('Illinois', 'Missouri'): 0, ('Illinois', 'Kentucky'): 11, ('Illinois', 'Tennessee'): -8, ('Illinois', 'Ohio'): 15, ('Illinois', 'Minnesota'): -21, ('Illinois', 'West Virginia'): 95, ('Illinois', 'Michigan'): 36, ('Illinois', 'Oklahoma'): -14, ('Illinois', 'Washington'): -15, ('Illinois', 'New York'): 33, ('Illinois', 'Texas'): 20, ('Illinois', 'California'): -8, ('Illinois', 'Indiana'): 61, ('Illinois', 'Illinoi

In [ ]:
### 문제점  
# 시차를 계산하는 이유 --> 비행시간을 구하기 위해 --> 그렇다면 비행시간은 왜 구하는데? --> 거리와 비행시간이 비례하기 때문에 모든 데이터의 거리가 나와있으므로 거리 별 비행시간을 구하려고 
#--> 거리별 비행시간은 왜 구하는데 ? --> 모든 데이터의 비행시간을 구해서 도착.출발시간 결측치를 채우기 위해. 

## 시차를 구할 때 데이터의 종류가 많지 않으면?? 시차의 오류가 발생. 그렇다면 시차 계산을 할 때 데이터가 많은 경우만 적으면 되지 않을까??
# 어차피 시차를 구하는 것은 거리별 비행시간을 구하기 위함임. 비행시간은 도착예정시간-출발예정시간을 빼서 구했는데 도착예정시간이 도착 주의 시간을 따라 시차가 발생함. 
# 때문에 비행시간 = 도착예정시간 - 출발예정시간 - 시차 이런식의 계산을 통해  비행시간을 구하여 전체 평균 비행시간과 전체 평균 거리를 통해  거리 별 비행 시간을 구하여 결측치가 존재하는 데이터의 비행시간도 구할 수 잇음.
# 그렇다면 굳이  모든 데이터의 시차를 구하는 것이 아닌 필요한 데이터마 구해도 되지 않을가 .... 